In [164]:
from IPython.display import clear_output
%cd /content/drive/MyDrive/collab_sandbox/learn_DL/dive_into_dl/
# !pip install colabcode
clear_output()

# Pandas

https://pandas.pydata.org/docs/user_guide/10min.html#setting

# Titanic Kaggle

https://www.kaggle.com/c/titanic

In [165]:
import pandas as pd
import torch
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [217]:
dataset = pd.read_csv("./data/TitanicKaggle/train.csv")
# dataset_test = pd.read_csv("./data/TitanicKaggle/test.csv")
dataset_train, dataset_test = train_test_split(dataset, test_size=0.33, random_state=42)

dataset_train = dataset_train.reset_index()
dataset_test = dataset_test.reset_index()

drop_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin', 'index']
dataset_train = dataset_train.drop(columns=drop_columns)
dataset_test = dataset_test.drop(columns=drop_columns)

In [218]:
dataset_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,1,male,54.0,0,0,51.8625,S
1,0,3,male,NaN,0,0,15.5000,Q
2,0,2,male,25.0,1,2,41.5792,C
3,0,3,male,26.0,1,0,14.4542,C
4,0,3,female,22.0,0,0,10.5167,S


In [221]:
def encode_categorical(dataset, columns, full_dataset):
  dataset = dataset.copy(deep=True)
  # print(dataset)
  dataset_cols = dataset.columns.values
  for column in columns:
    if column in dataset_cols:
      values_list = full_dataset[column].values.tolist()
      list_columns = sorted(list(map(str, list(set(values_list)))))
      list_columns_renamed = [f'{column}_{item}' for item in list_columns]
      dict_columns = {item: i for (i, item) in enumerate(list_columns)}
      
      # print(dataset[column].)
      dataset_values = dataset[column].tolist()
      for i, value in enumerate(dataset_values):
        value = str(value)

        ohe_hot = [0 for _ in range(len(list_columns))]
        ohe_hot[dict_columns.get(value)] = 1
        dataset.loc[i, list_columns_renamed] = ohe_hot
      dataset = dataset.drop(columns=[column],) 
  
  return dataset

# encode_categorical(dataset_train, ['Sex', 'Embarked'], dataset).head(3)

In [220]:
categorical_labels = ['Pclass', 'SibSp',  'Embarked', 'Sex', 'Parch',]
dataset_train = encode_categorical(dataset_train, categorical_labels, dataset)
dataset_test = encode_categorical(dataset_test, categorical_labels, dataset)
dataset_train.head(5)

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,54.0,51.8625,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,NaN,15.5000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25.0,41.5792,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0,26.0,14.4542,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,22.0,10.5167,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
len(dataset_train.columns) == len(dataset_test.columns)

True

In [200]:
dataset_test.head(5)

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,1,NaN,15.2458,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,31.0,10.5000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,20.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6.0,33.0000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,14.0,11.2417,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
def standardize_data(dataset, exclude_columns,):
  dataset = dataset.copy(deep=True)
  dataset_cols = list(set(dataset.columns.values) - set(exclude_columns))
  for column_name in dataset_cols:
    column = dataset[column_name].to_numpy()
    nan_indexes = np.isnan(column)
    real_indexes = ~np.isnan(column)

    n = real_indexes.astype(int).sum()
    real_column = column[real_indexes]

    real_column_mean = real_column.sum() / n # or .mean()  
    real_column_var = (np.sum([(column[i]-real_column_mean)**2 if item else 0 for i, item in enumerate(list(real_indexes))]) / n)**(1/2)
    std_column = np.array([(column[i]-real_column_mean) / real_column_var if item else 0 for i, item in enumerate(list(real_indexes))])
    
    if np.isnan(std_column).any():
      std_column = np.zeros_like(std_column)
    
    dataset.loc[:, column_name] = std_column 
  return dataset

dataset_train = standardize_data(dataset_train, ['Survived'])
dataset_test = standardize_data(dataset_test, ['Survived'])
dataset_train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,1.694606,0.387842,1.847922,-0.515163,-1.136797,0.694685,-0.548917,-0.186339,-0.137126,-0.149327,-0.09198,-0.109016,-0.462438,-0.312419,0.603195,-0.040996,-0.726777,0.726777,0.551505,-0.385178,-0.309173,-0.071127,-0.071127,-0.071127,-0.040996
1,0,0.000000,-0.319081,-0.541148,-0.515163,0.879664,0.694685,-0.548917,-0.186339,-0.137126,-0.149327,-0.09198,-0.109016,-0.462438,3.200825,-1.657838,-0.040996,-0.726777,0.726777,0.551505,-0.385178,-0.309173,-0.071127,-0.071127,-0.071127,-0.040996
2,0,-0.313384,0.187924,-0.541148,1.941134,-1.136797,-1.439502,1.821769,-0.186339,-0.137126,-0.149327,-0.09198,-0.109016,2.162450,-0.312419,-1.657838,-0.040996,-0.726777,0.726777,-1.813221,-0.385178,3.234430,-0.071127,-0.071127,-0.071127,-0.040996
3,0,-0.244143,-0.339412,-0.541148,-0.515163,0.879664,-1.439502,1.821769,-0.186339,-0.137126,-0.149327,-0.09198,-0.109016,2.162450,-0.312419,-1.657838,-0.040996,-0.726777,0.726777,0.551505,-0.385178,-0.309173,-0.071127,-0.071127,-0.071127,-0.040996
4,0,-0.521107,-0.415961,-0.541148,-0.515163,0.879664,0.694685,-0.548917,-0.186339,-0.137126,-0.149327,-0.09198,-0.109016,-0.462438,-0.312419,0.603195,-0.040996,1.375937,-1.375937,0.551505,-0.385178,-0.309173,-0.071127,-0.071127,-0.071127,-0.040996


In [224]:
dataset_test.head(5)

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,1,0.000000,-0.382533,-0.615227,-0.5,0.950382,-1.521383,1.776212,-0.166957,-0.131306,-0.131306,0.0,0.0,1.918994,-0.297592,-1.533712,-0.058321,-0.759961,0.759961,-1.728145,2.489000,-0.323835,-0.082619,-0.058321,-0.082619,0.0
1,0,0.064822,-0.485993,-0.615227,2.0,-1.052209,0.657297,-0.562996,-0.166957,-0.131306,-0.131306,0.0,0.0,-0.521106,-0.297592,0.652013,-0.058321,-0.759961,0.759961,0.578655,-0.401768,-0.323835,-0.082619,-0.058321,-0.082619,0.0
2,0,-0.685579,-0.542129,-0.615227,-0.5,0.950382,0.657297,-0.562996,-0.166957,-0.131306,-0.131306,0.0,0.0,-0.521106,-0.297592,0.652013,-0.058321,-0.759961,0.759961,0.578655,-0.401768,-0.323835,-0.082619,-0.058321,-0.082619,0.0
3,1,-1.640635,0.004513,-0.615227,2.0,-1.052209,0.657297,-0.562996,-0.166957,-0.131306,-0.131306,0.0,0.0,-0.521106,-0.297592,0.652013,-0.058321,1.315858,-1.315858,-1.728145,2.489000,-0.323835,-0.082619,-0.058321,-0.082619,0.0
4,1,-1.094889,-0.469824,-0.615227,-0.5,0.950382,-1.521383,1.776212,-0.166957,-0.131306,-0.131306,0.0,0.0,1.918994,-0.297592,-1.533712,-0.058321,1.315858,-1.315858,0.578655,-0.401768,-0.323835,-0.082619,-0.058321,-0.082619,0.0


In [34]:
# some.drop(labels=['Survived'])

https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [246]:
class LinearRegressionDirty(nn.Module):
  def __init__(self, 
                features=None,
                targets=None
    ):
    super(LinearRegressionDirty, self).__init__()
    self.W = torch.nn.Parameter(
        torch.normal(0, 0.01, 
        size=(features, targets), 
        requires_grad=True))
    
    self.b = torch.nn.Parameter(
        torch.normal(0, 0.01,
        size=(1,targets),
        requires_grad=True))
    
  @staticmethod
  def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

  def softmax(self, x):
    x = torch.exp(x)
    x_sum = x.sum(1, keepdim=True)
    return x / x_sum 

  def forward(self, x):
    x = torch.matmul(x, self.W) + self.b
    x = self.softmax(x)
    return x

class CustomSGD:
  def __init__(self, 
               params=None, 
               lr=None
    ):
    self.lr = lr
    self.params = params
  
  def step(self):
    # print("Step")
    with torch.no_grad():
      for param in self.params:
        # print(param.grad)
        param -= self.lr * param.grad

  def zero_grad(self):
    for param in self.params:
      param.grad.zero_()

model = LinearRegressionDirty(features=3, targets=2)
print(model(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [11, 12, 13]], dtype=torch.float32)))
predicts = model(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [11, 12, 13]], dtype=torch.float32))
loss = nn.CrossEntropyLoss()(predicts, torch.tensor([0, 1, 0, 1]))
print(loss)
loss.backward()
opt = CustomSGD(params=model.parameters(), lr=0.01)

opt.step()

tensor([[0.5158, 0.4842],
        [0.5334, 0.4666],
        [0.5508, 0.4492],
        [0.5739, 0.4261]], grad_fn=<DivBackward0>)
tensor(0.7045, grad_fn=<NllLossBackward0>)


https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class CustomPandasDataset(Dataset):
  def __init__(self, 
                dataset=None,
                target_columns=None
      ):
      self.dataset = dataset
      self.target_columns = target_columns

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      data_row = self.dataset.iloc[idx]
      target = torch.tensor(data_row[self.target_columns].values, dtype=torch.long).squeeze(0)
      data_row = data_row.drop(labels=self.target_columns)
      features = torch.tensor(data_row.values, dtype=torch.float32)

      return features, target 

temp_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=['Survived']
)

temp_dataloader = DataLoader(
    dataset=temp_dataset, 
    batch_size=16, 
    shuffle=True
)

next(iter(temp_dataloader))
# temp_dataset

In [ ]:
dataset_train.iloc[0][['Survived']].values

In [256]:
class Trainer:
  def __init__(self, 
               model=None, 
               train_dataloader=None,
               test_dataloader=None,
               optimizer_class=None,
               loss_func=None,
    ): 
    self.model = model
    self.train_dataloader = train_dataloader
    self.test_dataloader = test_dataloader
    self.optimizer_class = optimizer_class
    self.optimizer = None
    self.loss_func = loss_func

    self.unit_tests()

  def train(self, 
            is_unit_test=False,
            epochs=1,
            lr=0.01
    ):
    self.optimizer = self.optimizer_class(
        params=self.model.parameters(),
        lr=lr
    )
    for epoch in range(epochs):
      print(f"Epoch {epoch}:")
      total_examples = 0
      total_loss = 0
      for features, targets in self.train_dataloader:
        predicts = self.model(features)
        loss = self.loss_func(predicts, targets)
        total_loss += loss
        total_examples += len(targets)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        
        if is_unit_test:
          break
      print(f"train loss => {total_loss / total_examples}")
      
      total_examples = 0
      total_loss = 0
      total_acc = 0
      with torch.no_grad():
        for features, targets in self.test_dataloader:
          predicts = self.model(features)
          loss = self.loss_func(predicts, targets)
          total_loss += loss
          predicts = predicts.argmax(-1)
          total_examples += len(targets)
          total_acc += (predicts == targets).type(torch.float32).sum()

      print(f"test loss => {total_loss / total_examples}")
      print(f"test acc => {total_acc / total_examples}")
      
      if is_unit_test:
        break

  
  def unit_tests(self):
    print("run unit tests")
    self.train(
        is_unit_test=True,
        epochs=1,
        lr=0.01
    )
    print("Ok")


train_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=['Survived']
)
train_dataloader = DataLoader(
    dataset=train_dataset, 
    batch_size=16, 
    shuffle=True
)

test_dataset = CustomPandasDataset(
    dataset=dataset_test,
    target_columns=['Survived']
)
test_dataloader = DataLoader(
    dataset=test_dataset, 
    batch_size=16, 
    shuffle=True
)

# print(train_dataset[0])
features_size = len(train_dataset[0][0])
targets_size = 2 #len(train_dataset[0][1])
model=LinearRegressionDirty(
    features=features_size, 
    targets=targets_size
)

# optimizer_class = CustomSGD
optimizer_class = torch.optim.Adam
loss_func = nn.CrossEntropyLoss()
# loss_func = LinearRegressionDirty.cross_entropy
trainer = Trainer(
  model=model, 
  train_dataloader=train_dataloader,
  test_dataloader=test_dataloader,
  optimizer_class=optimizer_class,
  loss_func=loss_func
)

run unit tests
Epoch 0:
train loss => 0.043287016451358795
test loss => 0.044398363679647446
test acc => 0.5525423884391785
Ok


In [254]:
trainer.train(epochs=10)

Epoch 0:
train loss => 0.03810993954539299
test loss => 0.034648120403289795
test acc => 0.7932203412055969
Epoch 1:
train loss => 0.03330230712890625
test loss => 0.03339386731386185
test acc => 0.7898305058479309
Epoch 2:
train loss => 0.032698214054107666
test loss => 0.03302357718348503
test acc => 0.7932203412055969
Epoch 3:
train loss => 0.03217019513249397
test loss => 0.03325984999537468
test acc => 0.800000011920929
Epoch 4:
train loss => 0.031910140067338943
test loss => 0.033185362815856934
test acc => 0.7932203412055969
Epoch 5:
train loss => 0.032210275530815125
test loss => 0.033011432737112045
test acc => 0.800000011920929
Epoch 6:
train loss => 0.031907740980386734
test loss => 0.03247613459825516
test acc => 0.806779682636261
Epoch 7:
train loss => 0.03141534700989723
test loss => 0.03277800232172012
test acc => 0.800000011920929
Epoch 8:
train loss => 0.0316154770553112
test loss => 0.03267255052924156
test acc => 0.803389847278595
Epoch 9:
train loss => 0.03185107558